In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

**keys**: electrostatics, homogeneous Dirichlet bvp, single layer potential $V$

Dirichlet Laplace Indirect Method
=============================

We consider the Dirichlet boundary value problem 

$$ \left\{ \begin{array}{rcl l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_0 u&=& m, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

Choosing an indirect ansatz the solution $u\in H^1(\Omega)$ is given by

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}s_y }_{\displaystyle{ \mathrm{LaplaceSL}(j) } }\,,$$ 

where $\mathrm j $ solves the boundary integral equation 

$$ \mathrm V\,\mathrm j = \mathrm M\,\mathrm m\quad \text{on} \; \Gamma\,.$$


Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(3)

Create test and trial function finite element spaces for $H^{-\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()

**Boundary Integral Equation**

We carefully apply the Dirichlet trace on the ansatz for $u$ and get an integral equation for the unknown density $j\in H^{-\frac12}(\Gamma)$. The integral equation may be read as follows: determine $j$ such that 

$$\begin{array}{r c ccc} \forall \, v\in H^{-\frac12}(\Gamma): && \displaystyle \int\limits_\Gamma \gamma_0 \left(\mathrm{SL}(j)\right) \cdot v \, \mathrm d s &=& \displaystyle \int\limits_\Gamma m \cdot v \, \mathrm d s \\[2ex] & \Rightarrow & \mathrm{V}\,\mathrm{j} &=& \mathrm{M} \, \mathrm m \,. \end{array} $$ 
 

**Details**
  
For any $u_j, v_i \in H^{-\frac12}(\Gamma)$, the single layer potential operator is implemented as follows 

$$ V_{ij} = \langle v_i, \gamma_0 \mathrm{SL}(u_j) \rangle_{-\frac12} = \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma \frac{1}{\|x-y\|} \, u_j(x) \, v_i(y) \, \mathrm{d} s_{y} \, \mathrm{d} s_x $$ 

You may assemble $V$ it by

```
V = LaplaceSL(u*ds)*v*ds
```

Compute the given Dirichlet data $m$, the right hand side and the matrix $\mathrm V$ (single layer potential operator).

In [ ]:
m = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
rhs = LinearForm (m*v.Trace()*ds(bonus_intorder=3)).Assemble()
with TaskManager(pajetrace=1000*1000*1000):
    V = LaplaceSL(u*ds)*v*ds   

Solve the linear system:

In [ ]:
j = GridFunction(fesL2)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse()
with TaskManager(pajetrace=1000*1000*1000):  
    CG(mat = V.mat, pre=pre, rhs = rhs.vec, sol=j.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)
Draw (j);

In [ ]:
type(V)
#for t in Timers():
#    if "ngbem" in t["name"]:
#        print (t)

In [ ]:
screen = WorkPlane(Axes( (0,0,0), Z, X)).RectangleC(0.5,0.5).Face()
screen.faces.name="screen"
vismesh = screen.GenerateMesh(maxh=0.05)
#Draw (vismesh);

In [ ]:
SLPotential = LaplaceSL( u*ds)
repformula = SLPotential(j)
fes_screen = H1(vismesh, order=5)
gf_screen = GridFunction(fes_screen)
with TaskManager():
    gf_screen.Set(repformula, definedon=vismesh.Boundaries("screen"))
Draw(repformula,vismesh)

**References**
- For details on the analysis of boundary integral equations derived from elliptic partial differential equations, see for instance [Strongly Elliptic Systems and Boundary Integral Equations](https://www.cambridge.org/de/universitypress/subjects/mathematics/differential-and-integral-equations-dynamical-systems-and-co/strongly-elliptic-systems-and-boundary-integral-equations?format=HB&isbn=9780521663328).
- The integration of singular pairings is done as proposed in [Randelementmethoden](https://link.springer.com/book/9783519003687)
- The adaptive cross approximation is done as proposed in [Hierarchical Matrices](https://link.springer.com/book/10.1007/978-3-540-77147-0).

